# Bingham Model: Yield Stress Fluid Fitting

Demonstrates Bingham plastic model fitting to flow curve data with yield stress detection.

## Learning Objectives

- Fit Bingham model to flow curve data (σ vs γ̇)
- Detect and interpret yield stress
- Understand viscoplastic behavior
- Bayesian inference for yield stress uncertainty

**Prerequisites:** Basic rheology, complete 01-maxwell-fitting.ipynb

**Estimated Time:** 30-35 minutes

In [ ]:
# Google Colab Setup - Run this cell first!
# Skip if running locally with rheojax already installed

import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Install rheojax and dependencies
    !pip install -q rheojax
    
    # Colab uses float32 by default - we need float64 for numerical stability
    # This MUST be set before importing JAX
    import os
    os.environ['JAX_ENABLE_X64'] = 'true'
    
    print("✓ RheoJAX installed successfully!")
    print("✓ Float64 precision enabled")

## Setup

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

import time
import warnings

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from rheojax.models.bingham import Bingham

from rheojax.core.data import RheoData
from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.pipeline.base import Pipeline

jax, jnp = safe_import_jax()
verify_float64()
print("✓ JAX float64 enabled")

np.random.seed(42)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

# Suppress matplotlib backend warning in VS Code
warnings.filterwarnings('ignore', message='.*non-interactive.*')


## Bingham Theory

The Bingham model describes materials requiring minimum stress (yield stress) to flow:

**Constitutive Equation:**
$$\sigma(\dot{\gamma}) = \sigma_y + \eta_p \dot{\gamma} \quad \text{for } \sigma > \sigma_y$$
$$\dot{\gamma} = 0 \quad \text{for } \sigma \leq \sigma_y$$

**Parameters:**
- $\sigma_y$ = yield stress (Pa) - minimum stress to initiate flow
- $\eta_p$ = plastic viscosity (Pa·s) - constant viscosity above yield

**Applications:** Toothpaste, mayonnaise, drilling mud, concrete

In [ ]:
# True parameters
sigma_y_true = 50.0  # Pa
eta_p_true = 1.0     # Pa·s

print(f"True Parameters:")
print(f"  σ_y  = {sigma_y_true:.1f} Pa")
print(f"  η_p  = {eta_p_true:.2f} Pa·s")

# Shear rate array
gamma_dot = np.logspace(-1, 2, 40)  # 0.1 to 100 s⁻¹

# True stress
sigma_true = sigma_y_true + eta_p_true * gamma_dot

# Add noise
noise_level = 0.02
noise = np.random.normal(0, noise_level * sigma_true)
sigma_noisy = sigma_true + noise

print(f"\nData: {len(gamma_dot)} points, noise {noise_level*100:.1f}%")

In [ ]:
# Visualize flow curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(gamma_dot, sigma_noisy, 'o', alpha=0.7, label='Data')
ax1.plot(gamma_dot, sigma_true, '--', linewidth=2, alpha=0.4, label='True')
ax1.axhline(sigma_y_true, color='red', linestyle=':', linewidth=2, label=f'Yield σ_y={sigma_y_true} Pa')
ax1.set_xlabel('Shear Rate γ̇ (s⁻¹)', fontweight='bold')
ax1.set_ylabel('Shear Stress σ (Pa)', fontweight='bold')
ax1.set_title('Flow Curve', fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Viscosity
eta_apparent = sigma_noisy / gamma_dot
ax2.semilogx(gamma_dot, eta_apparent, 'o', alpha=0.7, label='Apparent η')
ax2.axhline(eta_p_true, color='red', linestyle=':', linewidth=2, label=f'Plastic η_p={eta_p_true} Pa·s')
ax2.set_xlabel('Shear Rate γ̇ (s⁻¹)', fontweight='bold')
ax2.set_ylabel('Apparent Viscosity (Pa·s)', fontweight='bold')
ax2.set_title('Shear-Thinning Due to Yield Stress', fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
display(fig)
plt.close(fig)


## Fitting (Linear Regression)

Bingham model is linear: σ = σ_y + η_p·γ̇, so we can use simple linear regression.

In [ ]:
model = Bingham()
model.fit(gamma_dot, sigma_noisy)

sigma_y_fit = model.parameters.get_value('sigma_y')
eta_p_fit = model.parameters.get_value('eta_p')

print("="*60)
print("FITTED PARAMETERS")
print("="*60)
print(f"σ_y = {sigma_y_fit:.2f} Pa (true: {sigma_y_true:.2f})")
print(f"η_p = {eta_p_fit:.3f} Pa·s (true: {eta_p_true:.3f})")
print(f"\nErrors: σ_y {abs(sigma_y_fit-sigma_y_true)/sigma_y_true*100:.2f}%, η_p {abs(eta_p_fit-eta_p_true)/eta_p_true*100:.2f}%")
print("="*60)

In [ ]:
# Visualize fit
sigma_pred = model.predict(gamma_dot)

fig = plt.figure(figsize=(10, 6))
plt.plot(gamma_dot, sigma_noisy, 'o', alpha=0.7, label='Data')
plt.plot(gamma_dot, sigma_true, '--', linewidth=2, alpha=0.4, label='True')
plt.plot(gamma_dot, sigma_pred, '-', linewidth=2.5, label='Fitted')
plt.axhline(sigma_y_fit, color='red', linestyle=':', linewidth=2, label=f'Yield σ_y={sigma_y_fit:.1f} Pa')
plt.xlabel('Shear Rate γ̇ (s⁻¹)', fontweight='bold')
plt.ylabel('Shear Stress σ (Pa)', fontweight='bold')
plt.title('Bingham Model Fit', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
display(fig)
plt.close(fig)

r2 = 1 - np.sum((sigma_noisy - sigma_pred)**2) / np.sum((sigma_noisy - np.mean(sigma_noisy))**2)
print(f"R² = {r2:.6f}")


## Bayesian Inference

Quantify yield stress uncertainty - critical for process design!

In [ ]:
print("Running Bayesian inference...\n")

result = model.fit_bayesian(
    gamma_dot, sigma_noisy,
    num_warmup=1000,
    num_samples=2000,
    num_chains=1,
    initial_values={
        'sigma_y': sigma_y_fit,
        'eta_p': eta_p_fit
    }
)

print("Complete!")

In [ ]:
summary = result.summary
diagnostics = result.diagnostics
credible = model.get_credible_intervals(result.posterior_samples, 0.95)

print("="*60)
print("POSTERIOR SUMMARY")
print("="*60)
print(f"\nσ_y = {summary['sigma_y']['mean']:.2f} ± {summary['sigma_y']['std']:.2f} Pa")
print(f"η_p = {summary['eta_p']['mean']:.3f} ± {summary['eta_p']['std']:.3f} Pa·s")
print(f"\n95% CI σ_y: [{credible['sigma_y'][0]:.2f}, {credible['sigma_y'][1]:.2f}] Pa")
print(f"95% CI η_p: [{credible['eta_p'][0]:.3f}, {credible['eta_p'][1]:.3f}] Pa·s")
print(f"\nR-hat: σ_y={diagnostics['r_hat']['sigma_y']:.4f}, η_p={diagnostics['r_hat']['eta_p']:.4f}")
print(f"ESS: σ_y={diagnostics['ess']['sigma_y']:.0f}, η_p={diagnostics['ess']['eta_p']:.0f}")
print("="*60)

print(f"\n✓ Yield stress uncertainty: ±{summary['sigma_y']['std']/summary['sigma_y']['mean']*100:.1f}%")

## Key Takeaways

1. **Yield Stress:** Minimum stress for flow - critical for pumping/processing
2. **Linear Above Yield:** Newtonian behavior once flowing (η_p constant)
3. **Bayesian Uncertainty:** Quantifies yield stress uncertainty for safety margins

## Next Steps

- **[05-power-law-fitting.ipynb](05-power-law-fitting.ipynb)**: Shear-thinning without yield stress
- **[advanced/01-multi-technique-fitting.ipynb](../advanced/01-multi-technique-fitting.ipynb)**: Combined fitting

In [ ]:
import sys

import rheojax

print(f"Python: {sys.version}")
print(f"Rheo: {rheojax.__version__}")